# 課題　毒キノコの判定　SVM

## １　今回のゴール

### SVMを用いて、毒キノコの判定を行う。

## ２　SVMとは何か

### サポートベクターマシン（SVM)は、分類にも回帰にも使える優れた教師あり学習のアルゴリズムである。２クラスへの分類や３クラス以上への分類も可能であり、カーネルを使うことで非線形な分類も可能である。サポートベクターマシンは、線形入力素子を利用して 2 クラスのパターン識別器を構成する手法である。訓練サンプルから、各データ点との距離が最大となるマージン最大化超平面を求めるという基準（超平面分離定理）で線形入力素子のパラメータを学習する。与えられたデータを線形に分離することが可能な（例えば、3次元のデータを2次元平面で完全に区切ることができる）場合を考えたとき、SVMは与えられた学習用サンプルを、もっとも大胆に区切る境目を学習する。 学習の結果得られた超平面は、境界に最も近いサンプル（サポートベクター）との距離（マージン）が最大となるパーセプトロン（マージン識別器）で定義される。 学習過程はラグランジュの未定乗数法とKKT条件を用いることにより、最適化問題の一種である凸二次計画問題で定式化される。SVMは学習データのノイズにも強く、分類性能が非常に高い。また、他のアルゴリズムに比して学習データ数もそれ程多くは必要としない。ただし、分類処理速度は他のアルゴリズムに比して遅くなる。そして、基本的には２クラスへの分類器となるため、多クラスへの分類を行うためには複数のSVM分類器を組み合わせる必要がある。



## ３　必要なライブラリーをImport

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn import svm, metrics, preprocessing, cross_validation
from sklearn import grid_search 
from sklearn import metrics #識別結果の表示用
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用

/Users/katouyoshitaka/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/katouyoshitaka/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## ４　データの取得

In [2]:
data = pd.read_csv("mushrooms.csv")

In [3]:
data.head(10)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [4]:
data.tail(10)

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
8114,p,f,y,c,f,m,a,c,b,y,...,y,c,c,p,w,n,n,w,c,d
8115,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,v,l
8116,p,k,y,n,f,s,f,c,n,b,...,k,p,w,p,w,o,e,w,v,l
8117,p,k,s,e,f,y,f,c,n,b,...,s,p,w,p,w,o,e,w,v,d
8118,p,k,y,n,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,d
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
8123,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,c,l


In [5]:
data.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124,...,8124,8124,8124,8124,8124,8124,8124,8124,8124,8124
unique,2,6,4,10,2,9,2,2,2,12,...,4,9,9,1,4,3,5,9,6,7
top,e,x,y,n,f,n,f,c,b,b,...,s,w,w,p,w,o,p,w,v,d
freq,4208,3656,3244,2284,4748,3528,7914,6812,5612,1728,...,4936,4464,4384,8124,7924,7488,3968,2388,4040,3148


In [6]:
data.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8124 entries, 0 to 8123
Data columns (total 23 columns):
class                       8124 non-null object
cap-shape                   8124 non-null object
cap-surface                 8124 non-null object
cap-color                   8124 non-null object
bruises                     8124 non-null object
odor                        8124 non-null object
gill-attachment             8124 non-null object
gill-spacing                8124 non-null object
gill-size                   8124 non-null object
gill-color                  8124 non-null object
stalk-shape                 8124 non-null object
stalk-root                  8124 non-null object
stalk-surface-above-ring    8124 non-null object
stalk-surface-below-ring    8124 non-null object
stalk-color-above-ring      8124 non-null object
stalk-color-below-ring      8124 non-null object
veil-type                   8124 non-null object
veil-color                  8124 non-null object
ring-number

## ５　前処理

### （１）データ変換

In [8]:
# データ変換
data = data.apply(LabelEncoder().fit_transform)

In [9]:
data.describe()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,...,8124.000000,8124.000000,8124.000000,8124.0,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,0.482029,3.348104,1.827671,4.504677,0.415559,4.144756,0.974151,0.161497,0.309207,4.810684,...,1.603644,5.816347,5.794682,0.0,1.965534,1.069424,2.291974,3.596750,3.644018,1.508616
std,0.499708,1.604329,1.229873,2.545821,0.492848,2.103729,0.158695,0.368011,0.462195,3.540359,...,0.675974,1.901747,1.907291,0.0,0.242669,0.271064,1.801672,2.382663,1.252082,1.719975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,3.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,...,1.000000,6.000000,6.000000,0.0,2.000000,1.000000,0.000000,2.000000,3.000000,0.000000
50%,0.000000,3.000000,2.000000,4.000000,0.000000,5.000000,1.000000,0.000000,0.000000,5.000000,...,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,2.000000,3.000000,4.000000,1.000000
75%,1.000000,5.000000,3.000000,8.000000,1.000000,5.000000,1.000000,0.000000,1.000000,7.000000,...,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,4.000000,7.000000,4.000000,2.000000
max,1.000000,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,...,3.000000,8.000000,8.000000,0.0,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


In [10]:
X = data.iloc[:,1:]

In [11]:
y = data.iloc[:, 0]

In [12]:
for col in X.columns:
    X[col].astype('category')

In [13]:
X.head(10)

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5,2,4,1,6,1,0,1,4,0,...,2,7,7,0,2,1,4,2,3,5
1,5,2,9,1,0,1,0,0,4,0,...,2,7,7,0,2,1,4,3,2,1
2,0,2,8,1,3,1,0,0,5,0,...,2,7,7,0,2,1,4,3,2,3
3,5,3,8,1,6,1,0,1,5,0,...,2,7,7,0,2,1,4,2,3,5
4,5,2,3,0,5,1,1,0,4,1,...,2,7,7,0,2,1,0,3,0,1
5,5,3,9,1,0,1,0,0,5,0,...,2,7,7,0,2,1,4,2,2,1
6,0,2,8,1,0,1,0,0,2,0,...,2,7,7,0,2,1,4,2,2,3
7,0,3,8,1,3,1,0,0,5,0,...,2,7,7,0,2,1,4,3,3,3
8,5,3,8,1,6,1,0,1,7,0,...,2,7,7,0,2,1,4,2,4,1
9,0,2,9,1,0,1,0,0,2,0,...,2,7,7,0,2,1,4,2,3,3


In [14]:
X.tail(10)

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
8114,2,3,1,0,4,0,0,0,11,0,...,3,1,1,0,2,0,3,7,1,0
8115,5,2,4,0,5,0,0,0,11,0,...,2,5,5,0,1,1,4,4,4,2
8116,3,3,4,0,7,1,0,1,0,1,...,1,6,7,0,2,1,0,7,4,2
8117,3,2,2,0,8,1,0,1,0,1,...,2,6,7,0,2,1,0,7,4,0
8118,3,3,4,0,2,1,0,1,0,1,...,2,6,7,0,2,1,0,7,4,0
8119,3,2,4,0,5,0,0,0,11,0,...,2,5,5,0,1,1,4,0,1,2
8120,5,2,4,0,5,0,0,0,11,0,...,2,5,5,0,0,1,4,0,4,2
8121,2,2,4,0,5,0,0,0,5,0,...,2,5,5,0,1,1,4,0,1,2
8122,3,3,4,0,8,1,0,1,0,1,...,1,7,7,0,2,1,0,7,4,2
8123,5,2,4,0,5,0,0,0,11,0,...,2,5,5,0,1,1,4,4,1,2


In [15]:
X.describe()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
count,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,...,8124.000000,8124.000000,8124.000000,8124.0,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000,8124.000000
mean,3.348104,1.827671,4.504677,0.415559,4.144756,0.974151,0.161497,0.309207,4.810684,0.567208,...,1.603644,5.816347,5.794682,0.0,1.965534,1.069424,2.291974,3.596750,3.644018,1.508616
std,1.604329,1.229873,2.545821,0.492848,2.103729,0.158695,0.368011,0.462195,3.540359,0.495493,...,0.675974,1.901747,1.907291,0.0,0.242669,0.271064,1.801672,2.382663,1.252082,1.719975
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,3.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,0.000000,...,1.000000,6.000000,6.000000,0.0,2.000000,1.000000,0.000000,2.000000,3.000000,0.000000
50%,3.000000,2.000000,4.000000,0.000000,5.000000,1.000000,0.000000,0.000000,5.000000,1.000000,...,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,2.000000,3.000000,4.000000,1.000000
75%,5.000000,3.000000,8.000000,1.000000,5.000000,1.000000,0.000000,1.000000,7.000000,1.000000,...,2.000000,7.000000,7.000000,0.0,2.000000,1.000000,4.000000,7.000000,4.000000,2.000000
max,5.000000,3.000000,9.000000,1.000000,8.000000,1.000000,1.000000,1.000000,11.000000,1.000000,...,3.000000,8.000000,8.000000,0.0,3.000000,2.000000,4.000000,8.000000,5.000000,6.000000


In [16]:
y.head(10)

0    1
1    0
2    0
3    1
4    0
5    0
6    0
7    0
8    1
9    0
Name: class, dtype: int64

In [17]:
y.tail(10)

8114    1
8115    0
8116    1
8117    1
8118    1
8119    0
8120    0
8121    0
8122    1
8123    0
Name: class, dtype: int64

In [18]:
y.describe()

count    8124.000000
mean        0.482029
std         0.499708
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: class, dtype: float64

### （２）データセットの分割

In [19]:
# データセットの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [20]:
X.shape

(8124, 22)

In [21]:
X_train.shape

(5686, 22)

In [22]:
X_test.shape

(2438, 22)

In [23]:
y[:,np.newaxis].shape

(8124, 1)

In [24]:
y_train[:,np.newaxis].shape

(5686, 1)

In [25]:
y_test[:,np.newaxis].shape

(2438, 1)

### （３）標準化

In [26]:
# 標準化
stdsc = StandardScaler()

# 訓練用のデータのみの平均、標準偏差を用いて標準化する
X_train_std = stdsc.fit_transform(X_train)
# テストデータも標準化
X_test_std = stdsc.transform(X_test)

print(X_train_std.mean())
print(X_train_std.std())
# test用はtrainを基準に標準化してるので、多少0,1からずれる
print(X_test_std.mean())
print(X_test_std.std())

-1.1076314510377924e-17
0.9770084209183945
-0.013399953276692139
0.9858136676832726


## ６　ハイパーパラメータの調整

### （１）カーネルとは何か

### 分類を行う際に、単純に直線では分類できない非線形な分類問題の場合に、高次元空間へと移して分類することをカーネルという。このカーネルには、高次元への移し方によって、線形カーネル（linear）、多項式カーネル（poly）、RBFカーネル（rbf）、シグモイドカーネル（sigmoid）等の手法がある。SVCのデフォルトにも設定され、最も広く利用されているカーネルがRBFカーネルである。ただ、RBFカーネルは、高い表現力ゆえに比較的過学習しやすいという点は注意が必要である。また、例えば、データ数と比べて特徴量の数が圧倒的に多いような場合は、線形カーネルがうまくいき易いといわれている。

### （２）コストペナルティCとは何か

### 分類を行なう際に、誤分類を少なくしつつマージンを広く取ることを目指すが、コストペナルティCは、誤分類されている度合いを示すパラメータになる。すなわち、誤分類の影響をどれだけ反映させるかをこのパラメータによって指定することができる。Cの値を大きくする程、誤分類の際のペナルティが大きくなる（すなわち、誤りに厳しくなる。）ので、Cの値を大きくすればする程、過学習を起こしやすくなる。データ数が大きくなると外れ値が一定数存在してしまうものなので、全てを綺麗に分類しようとせずに、汎用性を損なわない程度のコストに留める必要がある。逆に、Cの値が小さ過ぎても、誤分類に対して寛容になり過ぎてしまうので、この場合にも注意が必要である。

### （３）ハイパーパラメータを調整する

In [27]:
clf=svm.SVC(class_weight='balanced', random_state=0)
param_range=[0.01, 0.1, 1.0] #変化させるパラメータに設定する値たち
param_grid=[{'C':param_range,'kernel':['rbf', 'linear'], 'gamma':param_range}] #Cとカーネルとgammaを変化させて最適化させる

In [28]:
gs=grid_search.GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy', cv=10, n_jobs=-1)
gs_1=gs.fit(X_train_std,y_train)
print(gs_1.best_score_)
print(gs_1.best_params_)

1.0
{'C': 1.0, 'gamma': 0.1, 'kernel': 'rbf'}


## ７　SVMの実行

In [29]:
clf_gs_1=gs_1.best_estimator_
pred=clf_gs_1.predict(X_test_std)
ac_score=metrics.accuracy_score(y_test,pred)
print(ac_score) #テストデータの正答率
cnfmat=metrics.confusion_matrix(y_true=y_test,y_pred=pred )
print(cnfmat) #混合行列の表示
report=metrics.classification_report(y_true=y_test,y_pred=pred )
print(report) #適合率、再現率、F値の結果

1.0
[[1247    0]
 [   0 1191]]
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1247
          1       1.00      1.00      1.00      1191

avg / total       1.00      1.00      1.00      2438

